In [ ]:
import numpy as np
import os

import pygimli as pg
from pygimli.meshtools import appendTriangleBoundary, merge2Meshes
from pygimli.mplviewer import drawMesh
from pygimli.viewer import showMesh
from pygimli.manager import MeshMethodManager

In [ ]:
#Ici on commence par définir un maillage rectangulaire homogène

xmin, xmax = 0.0, 0.02 #On définit les limites en x du maillage
zmin, zmax = 0. , 0. #On définit les limites en z du maillage

dx = 0.01 #On définit le pas de la maille
xreg = np.arange(xmin, xmax + dx, dx, 'float') #On calcule les coordonées des mailles
zreg = np.arange(zmin, zmax + dx, dx, 'float')

mesh1 = pg.Mesh(2) #On crée un maillage vide 
mesh1.create2DGrid(xreg, zreg, 0) #On crée le maillage
for c in mesh1.cells(): #On marque les cellules
    c.setMarker(1)

print(mesh1) #On affiche le mombre de noeuds, cellules et lignes

#showMesh(mesh1, mesh1.cellMarkers(), logScale=False, label="Region marker") #On définit les paramètres d'affichage

showMesh(mesh1, mesh1.cellMarkers(), label="Region marker")
#drawMesh(ax,mesh1)

In [ ]:
#A présent on crée une zone grâce à un polygone

poly = pg.Mesh(2)  # empty 2d mesh
nStart = poly.createNode(0.0, 0.0, 0.0) #On crée un noeud de départ, on travaille en 2D donc le dernier terme vaut 0.0

nA = nStart #On définit le noeud de départ
for z in zreg[1:]: #On démarre de 1 et on se balade sur l'axe des x en créant un noeud à chaque fois
    nB = poly.createNode(xmax, z, 0.0)
    poly.createEdge(nA, nB) #On définit un côté entre le noeud précédemment crée et le nouveau
    nA = nB #On remplace le noeud de départ par le noeud nouvellement crée

z2 = 0.3 #On définit une altitude z2 
nA = poly.createNode(xmax, z2, 0.0) #On crée un noeud
poly.createEdge(nB, nA) #On fait le lien entre le dernier noeud crée et celui-là
nB = poly.createNode(xmin, z2, 0.0) #On crée un autre noeud en symétrique
poly.createEdge(nA, nB) #On fait le lien avec le noeud précédent
nC = poly.createNode(xmin, -0.8, 0.0)
poly.createEdge(nB, nC)
nD = poly.createNode(0.4, -0.8, 0.0)
poly.createEdge(nC, nD)
nE = poly.createNode(0.4, 0.0, 0.0)
poly.createEdge(nD, nE)
poly.createEdge(nE, nStart) #On ferme le polygone!

tri = pg.TriangleWrapper(poly) #On appelle la fonction triangle
tri.setSwitches('-pzeAfaq31') #On rentre tout un tas de commandes rigolotes pour générer les triangles
# Ici on a :
# p : planar straight line graph ==> fichier poly
# z : on démarre le comptage à 0
# A : assigne un attribut à chaque triangle qui indique à quel segment il appartient et est lié
# f : algorithme de triangulation (?)
# a : impose une surface contrainte pour chaque triangle on peut ajouter un nombre si on veut préciser
# q31 : impose que les triangles générés aient au minimun des angles de 20° On peut ajouter un nombre derrière pour préciser le nombre que l'o souhaite

In [ ]:
#A présent on génère le maillage hétérogène

mesh2 = pg.Mesh(2) #On appelle le second maillage autour du premier
tri.generate(mesh2) #On génère les triangles au sein du polygone précédemment crée

for cell in mesh2.cells(): #On génère les cellules de chaque maille
    cell.setMarker(2)

In [ ]:
#Fusion des deux maillages pour créer un maillage hybride
#mesh3 = merge2Meshes(mesh1, mesh2) #On peut faire fusionner les deux maillages ainsi crées en un seul

a=showMesh(mesh2, mesh1.cellMarkers(), label="Region marker")

In [ ]:
#Test à effacer
#pg.meshtools.writePLC(mesh2, 'Test')
#read2d = pg.meshtools.readPLC('Test')
#print(read2d)

In [ ]:
pg_pos = mesh2.positions()
mesh_pos = np.array((np.array(pg.x(pg_pos)), np.array(pg.y(pg_pos)), np.array(pg.z(pg_pos)))).T #On crée une matrice contenant la position des noeuds
mesh_cells = np.zeros((mesh2.cellCount(), 3)) #Matrice vide de la taille du nombre de cellules
for i, cell in enumerate(mesh2.cells()): #On rentre les cellules das une matrice
        mesh_cells[i] = cell.ids()


In [ ]:
print(mesh_pos) #Permet d'afficher la position des noeuds

In [24]:
mesh2.boundaries() #utiliser la tabulation pour lister les operations possibles sur un objet. mesh.'tab par exemple
mesh2.cellCount()
a=np.array([1,2])
np.cross(a,a)

array(0)

In [25]:
for c in mesh2.cells(): #Afficher les coordonnées des noeuds pour chaque cellule
    print([[node.x(),node.y()] for node in c.nodes()])

[[0.010753377629, -0.285], [0.0, -0.29], [0.008863965212, -0.295]]
[[0.150177441689, -0.633557928406], [0.179813355773, -0.432295126134], [0.051680360248, -0.471568379339]]
[[0.051680360248, -0.471568379339], [-0.02, -0.425], [-0.02, -0.55]]
[[0.012995955939, -0.312682152341], [0.0, -0.3], [-0.005421352026, -0.318555334704]]
[[-0.005421352026, -0.318555334704], [-0.02, -0.33125], [0.006422196202, -0.346875]]
[[0.017127946667, -0.245], [0.037970645029, -0.248277120034], [0.041354972884, -0.227451560841]]
[[0.010753377629, -0.205], [0.0, -0.19999999999999998], [0.0, -0.21]]
[[0.0, -0.27999999999999997], [0.0, -0.29], [0.010753377629, -0.285]]
[[0.0, -0.09999999999999999], [0.0, -0.11], [0.017127946667, -0.105]]
[[0.008863965212, -0.265], [0.010753377629, -0.255], [0.0, -0.26]]
[[0.0, -0.27], [0.0, -0.27999999999999997], [0.017127946667, -0.275]]
[[0.047297333058999996, -0.155474960878], [0.070229495442, -0.158939476962], [0.07519653643, -0.136285218045]]
[[0.0, -0.27999999999999997], [0.

In [27]:
#c=mesh.cell(0) renvoie la cellule 0

def crossprod(c): #Fonction pour vérifier sur les triangles sont dans le bon ordre

    node0=c.node(0)
    a0=np.array([node0.x(),node0.y()])
    node1=c.node(1)
    a1=np.array([node1.x(),node1.y()])
    node2=c.node(2)
    a2=np.array([node2.x(),node2.y()])

    #faire un produit vectoriel entre node[1]-node[0] et node[2]-node[0]. 
    return np.cross(a1-a0,a2-a0)

crossprod(mesh2.cell(0))
# si négatif, node[1],node[2]=node[2],node[1]
# afficher node[0] node[1] node[2]
#c.boundaryNodes

array(9.808671420500008e-05)

In [28]:
f=open("maillage.txt","w") #On écrit la position des noeuds dans un fichier texte

for c in mesh2.cells(): 
    assert crossprod(c) > 0 # arrete tout si c'est faux.
    for node in c.nodes():
        #f.write(str([[node.x(),node.y()]]))
        f.write("{} {} \n".format(node.x(),node.y()))
f.close()

In [29]:
!cat maillage.txt

0.010753377629 -0.285 
0.0 -0.29 
0.008863965212 -0.295 
0.150177441689 -0.633557928406 
0.179813355773 -0.432295126134 
0.051680360248 -0.471568379339 
0.051680360248 -0.471568379339 
-0.02 -0.425 
-0.02 -0.55 
0.012995955939 -0.312682152341 
0.0 -0.3 
-0.005421352026 -0.318555334704 
-0.005421352026 -0.318555334704 
-0.02 -0.33125 
0.006422196202 -0.346875 
0.017127946667 -0.245 
0.037970645029 -0.248277120034 
0.041354972884 -0.227451560841 
0.010753377629 -0.205 
0.0 -0.19999999999999998 
0.0 -0.21 
0.0 -0.27999999999999997 
0.0 -0.29 
0.010753377629 -0.285 
0.0 -0.09999999999999999 
0.0 -0.11 
0.017127946667 -0.105 
0.008863965212 -0.265 
0.010753377629 -0.255 
0.0 -0.26 
0.0 -0.27 
0.0 -0.27999999999999997 
0.017127946667 -0.275 
0.047297333058999996 -0.155474960878 
0.070229495442 -0.158939476962 
0.07519653643 -0.136285218045 
0.0 -0.27999999999999997 
0.010753377629 -0.285 
0.017127946667 -0.275 
0.0 -0.11 
0.0 -0.12 
0.010753377629 -0.11499999999999999 
0.026936618087 -0.2932

In [21]:
#pg.x? #Permet d'avoir des infos grâce à ?

In [22]:
l=list(pg_pos)
l[1]
list(l[1])

In [31]:
mesh2.saveAscii('mesh.a')

1

In [101]:
#Création d'une fonction pour interpoler la charge au fond du trou

#Définition des paramètres initiaux
p = np.array(mesh_pos[:,:2]) #coordonnées xy des noeuds du maillage
d = 30 #profondeur du trou en cm
radius = 2 #rayon du trou en cm
h_0 = -95 #charge initiale en cm, soit l'état initial du sol (teneur en eau exprimée en charge)
h_1 = 10 #hauteur d'eau au fond du trou en cm


In [104]:
b = np.where(p[:,1]==0)


(array([  0,  30,  35,  39,  45,  66,  72,  78, 121, 132]),)


In [107]:
def charge_trou(p,d,radius,h_0,h_1):
    
    for i in range(1,len(p)) :
        if p[i,1]==d & p[i,0]<=radius :
            p[i,2]=1
            p[i,3]=h_1
        elif p[i,1]==0 :
            p[i,2]=-3
            p[i,3]=h_0
        else :
            p[i,3]=h_0

#Interpolation pour la valeur de charge au fond du trou
r = np.where((p[:,1]<=d+h_1)) & np.where(p[:,0]==radius)
for i in range(1,len(r)) :
    p[r(i),2] = 1
    p[r(i),3] = h_1-(p[r(i),1]-d)
        
        

TypeError: unsupported operand type(s) for &: 'tuple' and 'tuple'

In [89]:
mesh2.

138


In [100]:
g=np.where()

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [110]:
print(mesh2.cell(0))

<class 'pygimli.core._pygimli_.Triangle'>	ID: 0, Marker: 2, Size: 4.904335710250004e-05
	51 RVector3: (0.010753377629, -0.285, 0.0)
	1 RVector3: (0.0, -0.29, 0.0)
	36 RVector3: (0.008863965212, -0.295, 0.0)



In [119]:

c=mesh2.cell(0)
for node in c.nodes():
   print(node.i

ArgumentError: Python argument types in
    Node.pos(Node, int)
did not match C++ signature:
    pos(GIMLI::Node {lvalue})
    pos(GIMLI::Node {lvalue})